In [1]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import pandas as pd
import os, glob
import seaborn as sns
from scipy.stats import kruskal
import scikit_posthocs as sp
from scipy.stats import mannwhitneyu
from dotenv import load_dotenv, find_dotenv
from google.cloud import bigquery

# Automatically find and load the .env file
load_dotenv(find_dotenv())

project_id = os.getenv('GOOGLE_CLOUD_PROJECT')
client = bigquery.Client(project=project_id)
%load_ext google.cloud.bigquery

# load_dotenv('./Credentials.env',override=True)

# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] =str(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))
# os.environ['GCLOUD_PROJECT'] = str(os.getenv("GCLOUD_PROJECT"))

# %load_ext google.cloud.bigquery
# from google.cloud import bigquery
# client=bigquery.Client()
# from google.cloud import bigquery_storage_v1

/Users/sandychen/Desktop/Healthrex_workspace/scripts/Blood_Culture_Stewardship/blood_culture_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sandychen/Desktop/Healthrex_workspace/scripts/Blood_Culture_Stewardship/blood_culture_env/lib/python3.9/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
# from fancyimpute import IterativeImputer as FancyIterativeImputer
import pickle

## Data Prepration

In [ ]:
%%bigquery Train_set_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2015 and order_year<2022

In [ ]:
Train_set_df['ed_arrival_datetime'] = pd.to_datetime(Train_set_df['ed_arrival_datetime'])
Train_set_df['blood_culture_order_datetime'] = pd.to_datetime(Train_set_df['blood_culture_order_datetime'])
Train_set_df['earliest_iv_antibiotic_datetime'] = pd.to_datetime(Train_set_df['earliest_iv_antibiotic_datetime'])
Train_set_df['hours_between_ed_cult'] = (Train_set_df['blood_culture_order_datetime'] - Train_set_df['ed_arrival_datetime']).dt.total_seconds() / 3600
Train_set_df['hours_between_cult_abx'] = (Train_set_df['blood_culture_order_datetime'] - Train_set_df['earliest_iv_antibiotic_datetime']).dt.total_seconds() / 3600

In [ ]:
Identifiers=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']
Labels=['positive_blood_culture','positive_blood_culture_in_week']
Labs=['min_heartrate','max_heartrate', 'avg_heartrate', 'median_heartrate',
       'min_resprate', 'max_resprate', 'avg_resprate', 'median_resprate',
       'min_temp', 'max_temp', 'avg_temp', 'median_temp', 'min_sysbp',
       'max_sysbp', 'avg_sysbp', 'median_sysbp', 'min_diasbp',
       'max_diasbp', 'avg_diasbp', 'median_diasbp', 'min_wbc', 'max_wbc',
       'avg_wbc', 'median_wbc', 'min_neutrophils', 'max_neutrophils',
       'avg_neutrophils', 'median_neutrophils', 'min_lymphocytes',
       'max_lymphocytes', 'avg_lymphocytes', 'median_lymphocytes',
       'min_hgb', 'max_hgb', 'avg_hgb', 'median_hgb', 'min_plt',
       'max_plt', 'avg_plt', 'median_plt', 'min_na', 'max_na', 'avg_na',
       'median_na', 'min_hco3', 'max_hco3', 'avg_hco3', 'median_hco3',
       'min_bun', 'max_bun', 'avg_bun', 'median_bun', 'min_cr', 'max_cr',
       'avg_cr', 'median_cr', 'min_lactate', 'max_lactate', 'avg_lactate',
       'median_lactate', 'min_procalcitonin', 'max_procalcitonin',
       'avg_procalcitonin', 'median_procalcitonin']
Demos=[ 'gender','age']
ABX=['vanc', 'zosyn', 'vanc_zosyn', 'other_ABX']
Time_Varient_features=['hours_between_ed_cult', 'hours_between_cult_abx']
Diagnosis= ['bacteremia', 'septic_shock', 'infective_endocarditis',
       'septic_thrombophlebitis', 'vascular_graft_infection', 'CRBSI',
       'infectious_discitis', 'epidural_abscess', 'septic_arthritis',
       'meningitis', 'meningitis_bacteria', 'cholangitis',
       'bacterial_cholangitis', 'pyelonephritis',
       'acute_bacterial_pyelonephritis', 'severe_pneumonia',
       'acute_hematogenous_osteomyelitis', 'asplenia',
       'immunocompromised_state', 'severe_cellulitis', 'cystitis',
       'prostatitis', 'CAP', 'diabetic_foot_infection', 'colitis',
       'aspiration_pneumonia', 'uncomplicated_cholecystitis',
       'uncomplicated_diverticulitis', 'Uncomplicated_pancreatitis']

Feature_set=Identifiers+Labels+Labs+Demos+ABX+Diagnosis+Time_Varient_features (# select Features based on experiment)
Train_set_df=Train_set_df[Feature_set]

In [ ]:
Train_set_df.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'],inplace=True)

In [ ]:
Train_set_df['datapoint'] = Train_set_df.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']).ngroup() + 1

In [ ]:
Train_set_df['gender'] = Train_set_df['gender'].apply(lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))


In [ ]:
Train_set_df['Label']= (Train_set_df['positive_blood_culture']|Train_set_df['positive_blood_culture_in_week'])
X_train = Train_set_df.drop(columns=['positive_blood_culture', 'positive_blood_culture_in_week','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','Label'])
y_train = Train_set_df['Label']

In [ ]:
X_train['Fever'] = X_train['max_temp'].apply(lambda x: 1 if x > 100.4 else 0)

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
imputer.fit(X_train)  # Fit on the training data to calculate medians
X_train = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)

In [ ]:
from sklearn.preprocessing import StandardScaler
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(X_train)

# Transform the training, test, and validation data
X_train2 = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)

## Pointing Model

In [26]:
# Load the saved LR model
with open('logistic_regression_modelI.pkl', 'rb') as model_file:
    model_l2 = pickle.load(model_file)


In [27]:
# Get the coefficients
coefficients = model_l2.coef_[0]  # Coefficients for the features
intercept = model_l2.intercept_[0]  # Intercept term

# Print the coefficients and intercept
print("Coefficients:", coefficients)
print("Intercept:", intercept)

Coefficients: [ 0.20057557  0.00179579  0.33366419 -0.207842    0.19397862 -0.09343402
 -0.08960952 -0.21265723  0.1118055   0.09470426  0.02193637  0.20037391]
Intercept: -0.21577695703453317


In [28]:
coefficients = model_l2.coef_[0] 
# Get the feature names from your training data (assuming X_train2 is a DataFrame)
feature_names = X_train2.columns

# Identify non-zero coefficients and get corresponding feature names
non_zero_indices = np.where(coefficients != 0)[0]
non_zero_features = feature_names[non_zero_indices]
non_zero_coefficients = coefficients[non_zero_indices]

# Print the features and their corresponding non-zero coefficients
for feature, coef in zip(non_zero_features, non_zero_coefficients):
    print(f"Feature: {feature}, Coefficient: {coef}")

Feature: max_heartrate, Coefficient: 0.20057556921051362
Feature: max_resprate, Coefficient: 0.0017957877100702754
Feature: Fever, Coefficient: 0.33366419300147093
Feature: min_sysbp, Coefficient: -0.20784200224952826
Feature: max_wbc, Coefficient: 0.19397862134956015
Feature: min_na, Coefficient: -0.09343402422707259
Feature: min_hco3, Coefficient: -0.08960951907624168
Feature: min_plt, Coefficient: -0.21265723295675015
Feature: max_cr, Coefficient: 0.11180549512835596
Feature: max_lactate, Coefficient: 0.09470426443307634
Feature: gender, Coefficient: 0.021936369224784914
Feature: age, Coefficient: 0.20037390877239228


In [29]:
rounded_coefficients = np.round(non_zero_coefficients, 2)
rounded_intercept = np.round(intercept, 2)

# Create the logistic regression equation as a string
equation_terms = [f"{coef} * {name}" for coef, name in zip(rounded_coefficients, non_zero_features)]
equation = " + ".join(equation_terms)
full_equation = f"logit(P) = {rounded_intercept} + {equation}"

# Print the logistic regression equation
print("Logistic Regression Equation:")
print(full_equation)

Logistic Regression Equation:
logit(P) = -0.22 + 0.2 * max_heartrate + 0.0 * max_resprate + 0.33 * Fever + -0.21 * min_sysbp + 0.19 * max_wbc + -0.09 * min_na + -0.09 * min_hco3 + -0.21 * min_plt + 0.11 * max_cr + 0.09 * max_lactate + 0.02 * gender + 0.2 * age


## pointing system

In [32]:
coefficients = model_l2.coef_[0] 
# Get the feature names from your training data (assuming X_train2 is a DataFrame)
feature_names = X_train2.columns

# Identify non-zero coefficients and get corresponding feature names
non_zero_indices = np.where(coefficients != 0)[0]
non_zero_features = feature_names[non_zero_indices]
non_zero_coefficients = coefficients[non_zero_indices]

In [ ]:
import pandas as pd

# how do we decide the scaling factor?
# use coef and median as the threshold

def generate_feature_points_table(non_zero_features, coefficients, medians, scaling_factor=10):
    # Create a list to store the rows for the table
    table_data = []

    for feature, coef in zip(non_zero_features, coefficients):
        if 'min' in feature and coef>0:
                row = {
                'Feature': feature,
                'Condition': f'> {round(medians[feature],3)}',
                'Points': round(coef*scaling_factor)
                    }
        elif 'min' in feature and coef<0:
                row = {
                'Feature': feature,
                'Condition': f'< {round(medians[feature],3)}',
                'Points': round(-1*coef*scaling_factor)
                    }
        elif 'max' in  feature and coef>0:
                row = {
                'Feature': feature,
                'Condition': f'> {round(medians[feature],3)}',
                'Points': round(coef*scaling_factor)
                    }
        elif 'max' in  feature and coef<0:
                row = {
                'Feature': feature,
                'Condition': f'< {round(medians[feature],3)}',
                'Points': round(-1*coef*scaling_factor)
                    }
        else:
                row = {
                'Feature': feature,
                'Condition': f'> {round(medians[feature],3)}',
                'Points': round(coef*scaling_factor)
                    }
        # Add the row to the table data
        table_data.append(row)
    
    # Convert the list of rows into a DataFrame
    df_points_table = pd.DataFrame(table_data)
    return df_points_table

# Example usage
df_points_table = generate_feature_points_table(non_zero_features, coefficients, medians, scaling_factor=10)

# Display the table
print(df_points_table)

def save_table_as_image(df, filename="feature_points_table.png"):
    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(len(df.columns) * 2, len(df) * 0.4))  # Adjust size based on table content
    ax.axis('tight')
    ax.axis('off')

    # Create the table plot
    table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')

    # Adjust font size
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    
    # Adjust column width
    table.scale(1, 1.5)

    # Save the table as an image
    #plt.show()
    plt.savefig(filename, bbox_inches='tight', dpi=300)
    plt.close(fig)  # Close the figure after saving to prevent display


save_table_as_image(df_points_table[df_points_table.Points>0], "feature_points_table.png")


          Feature Condition  Points
0   max_heartrate   > 100.0       2
1    max_resprate    > 20.0       0
2           Fever     > 0.0       3
3       min_sysbp   < 113.0       2
4         max_wbc     > 9.9       2
5          min_na   < 136.0       1
6        min_hco3    < 24.0       1
7         min_plt   < 222.0       2
8          max_cr    > 0.96       1
9     max_lactate    > 1.43       1
10         gender     > 0.0       0
11            age    > 65.0       2


In [36]:
# Calculate sensitivity, specificity, PPV, and NPV
y_pred_optimal3 = (total_points >=6).astype(int)
tn3, fp3, fn3, tp3 = confusion_matrix(y_test, y_pred_optimal3).ravel()
sensitivity3 = tp3 / (tp3 + fn3)  # Sensitivity or Recall
specificity3 = tn3 / (tn3 + fp3)  # Specificity
ppv3 = tp3 / (tp3 + fp3)          # Positive Predictive Value (Precision)
npv3 = tn3 / (tn3 + fn3)          # Negative Predictive Value
print(sensitivity3)
print(specificity3)
print(ppv3)
print(npv3)


0.8896551724137931
0.33506472426246803
0.03787211555398978
0.9904042221422574
